In [1]:
from LowPrecisionApproxGP.model.inducing_point_kernel import VarPrecisionInducingPointKernel
import gpytorch
from gpytorch.kernels.inducing_point_kernel import InducingPointKernel
import torch
from LowPrecisionApproxGP.util.GreedyTrain import greedy_train
from LowPrecisionApproxGP.model.models import VarPrecisionModel
from LowPrecisionApproxGP import load_bikes, load_road3d

In [14]:
# Define Model
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)



In [15]:
# dtype = torch.float32
# train_x, train_y = train_x.to(dtype), train_y.to(dtype)
# # Create Likelihood / Model
# likelihood = gpytorch.likelihoods.GaussianLikelihood()
# model = ExactGPModel(train_x, train_y, likelihood, dtype)
# mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood,model)

# # Set to training mode
# model.train()
# likelihood.train()

# # Use the adam optimizer
# optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters
# # Train Model
# model = greedy_train((train_x,train_y), model, mll,max_inducing_points,training_iter,dtype=dtype,Use_Max=True,J=20,max_Js=100)

In [16]:
train, test = load_bikes()
train_x, train_y = train
# # Create Likelihood / Model
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = ExactGPModel(train_x, train_y, likelihood)
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood,model)

# Set to training mode
model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters
# Train Model


In [17]:
max_iter = 200
for i in range(max_iter):
    optimizer.zero_grad()
    mll.zero_grad()
    # Output from model
    output = model(train_x)

    # Calc average loss and backprop gradients
    loss = -mll(output, train_y)
    loss.mean().backward()

    print(
    f"Message:Iteration:{i+1}/{max_iter}, Average_Loss:{loss.mean().item()}"
    )
    torch.cuda.empty_cache()

    optimizer.step()

, Message:Iteration:1/200, Average_Loss:49155.90303754907
, Message:Iteration:2/200, Average_Loss:45745.34508727801
, Message:Iteration:3/200, Average_Loss:42649.18711158417
, Message:Iteration:4/200, Average_Loss:39840.85640701716
, Message:Iteration:5/200, Average_Loss:37294.67875391016
, Message:Iteration:6/200, Average_Loss:34987.523529762875
, Message:Iteration:7/200, Average_Loss:32897.47672513231
, Message:Iteration:8/200, Average_Loss:31004.469894043214
, Message:Iteration:9/200, Average_Loss:29289.3252570493
, Message:Iteration:10/200, Average_Loss:27733.82984946021
, Message:Iteration:11/200, Average_Loss:26321.366742178892
, Message:Iteration:12/200, Average_Loss:25036.933543962277
, Message:Iteration:13/200, Average_Loss:23867.031461482784
, Message:Iteration:14/200, Average_Loss:22799.47694993811
, Message:Iteration:15/200, Average_Loss:21823.235354062195
, Message:Iteration:16/200, Average_Loss:20928.383370297754
, Message:Iteration:17/200, Average_Loss:20106.05522533984
